In [1]:
import nltk
import ssl
import numpy as np
import random
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from textblob import TextBlob
import pandas as pd

In [2]:
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

In [3]:
f = open('corpus.txt','r')
text = f.read()
text = text.lower()
sent_tokens = nltk.sent_tokenize(text)
word_tokens = nltk.word_tokenize(text)

In [4]:
def LemTokens(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
lemmatizer = nltk.stem.WordNetLemmatizer()

In [5]:
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [6]:
user_greetings = ["hi", "hello", "good morning", "hey", "what's up"]
bot_greetings = ["Hello, how may I be of assistance?"]
user_gratitude = ["thank you", "thanks", "that was helpful"]
bot_gratitude = ["You're welcome! Is there anything else you need?", "Happy to help! Are there other questions that I could help with?"]
bot_exit_text = ["Thank you for using my services. Have a great day!", "Hope I was helpful. See you later :)", "Bye!"]

In [7]:
def respond(input_text):
    bot_message = ""
    sent_tokens.append(input_text)
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        bot_message = "Apologies, I cannot understand your question. Please rephrase your question and try again."
    else:
        bot_message = sent_tokens[idx]
    return bot_message

In [8]:
def extract_sentiment(text):
    processed_text = TextBlob(text)
    sentiment = processed_text.sentiment
    if sentiment.polarity < 0:
        return "negative"
    elif sentiment.polarity > 0:
        return "positive"
    else:
        return "neutral"
    
def get_language(text):
    processed_text = TextBlob(text)
    return processed_text.detect_language()

In [ ]:
languages = {"en":"English", "fr":"France", "es:Spanish", "la":"Latin"}

In [9]:
print("Hi there! You are now interacting with a chatbot.")
print("Enter a number based on what you want to do: \n1. Chat with the bot \n2. Analyze the sentiment and detect the language of the text\n")
choice = int(input())

if choice == 1:
    print("I can answer any questions you may have about machine intelligence. Type in your question. If want to exit, type bye.\n")
    
exit_status = False
while exit_status is False:
    input_text = input()
    input_text = input_text.lower()
    if input_text != 'bye':
        if choice == 1:
            if input_text in user_greetings:
                print(random.choice(bot_greetings))
            else:
                if input_text in user_gratitude:
                    print(random.choice(bot_gratitude))
                else:
                    print(respond(input_text))
                    sent_tokens.remove(input_text)
        elif choice == 2:
            print("Detected Language: ", languages[get_language(input_text)])
            if get_language(input_text) == "en":
                print("Detected Sentiment: ", extract_sentiment(input_text))
            else:
                print("Sentiment can only be detected for text in English")
        else:
            print("Invalid choice!\nOnly 1 and 2 are valid choices \nPlese try running the program again.")
            exit_status = True
    else:
        exit_status = True
        print(random.choice(bot_exit_text))

Hi there! You are now interacting with a chatbot. I can answer any questions you may have about machine intelligence. Type in your question. If want to exit, type bye.
Enter a number based on what you want to do: 
1. Chat with the chatbot 
2. Analyze the sentiment and detect the language of the text

2
I enjoy drinking coffee
I think your response had the following sentiment:  positive
Your response is in the following language:  en
Bon appetite
I think your response had the following sentiment:  neutral
Your response is in the following language:  fr
Hola
I think your response had the following sentiment:  neutral
Your response is in the following language:  es
quid pro quid
I think your response had the following sentiment:  neutral
Your response is in the following language:  la
chitti navvu
I think your response had the following sentiment:  neutral
Your response is in the following language:  te
I like going to the beach
I think your response had the following sentiment:  neutral


KeyboardInterrupt: Interrupted by user